<a href="https://colab.research.google.com/github/frankgama/CS111-Intro-to-Scientific-Computing/blob/main/MidtermSummer2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Midterm COMPSCI 111 UCSB Summer 2025, Frank Gama
#I confirm that I did not use ChatGPT
#or codes from the web or from past years’ assignments and that the work
#I submit is my own and my own only.

#I have been using Google colab since this is the easiest for me at the moment

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
%matplotlib inline
np.set_printoptions(precision = 4)

#I have to connect my colab to my drive every time I make a project
from google.colab import drive
drive.mount('/content/drive/')

# Now that I am connected, I need to be able to access the CS111 File and its
# contents
import os, sys, pathlib
PKG_NAME = "CS111"
PKG_DIR = f"/content/drive/MyDrive/{PKG_NAME}"
PARENT = os.path.dirname(PKG_DIR)




#this makes sure I connected correctly and that my kernel has access to my files
if PARENT not in sys.path:
    sys.path.insert(0, PARENT)

import cs111.LU as LU


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#1.1 Show cs111.LUfactor(A) works for the transpose of A for Ax = B.
#We know that we get A[p,:] = LU, or more generally, PA = LU. We know that
#P^-1 = P^T. so we can rewrite A = P^-1LU. This just means that if we take the
#the permutation and apply it to L and U we get back the original matrix A.
#now we that in (AB)^T = B^T A^T, so we apply this to and get
# A^T = U^T L^T P. Thus we are able to use our functions without changing much.


In [ ]:
def LUsolve_withTranspose(A, b):

#get L, U and P
  L, U, P = LU.LUfactor(A, pivoting = True)

  #solve for U^T y = b
  y = LU.Lsolve(L, b)

  #solve for x = L^T x = y
  x = LU.Usolve(U, y)

  a = np.zeros_like(x)

  x[P] = a

  rel_res = np.linalg.norm(b - A.T @ a)/ np.linalg.norm(b)

  return a, rel_res



In [ ]:
rng = np.random.default_rng(12345)
A5 = rng.standard_normal((5, 5))
A5 += 5.0 * np.eye(5)
b5 = rng.standard_normal(5)

x5, rel_res5 = LUsolve_withTranspose(A5, b5)
print("\n5x5 Solution:", x5)
print("5x5 Relative residual:", rel_res5)


xref5 = npla.solve(A5.T, b5)
rel_err5 = npla.norm(x5 - xref5) / npla.norm(xref5)
print("5x5 Relative error vs NumPy:", rel_err5)


5x5 Solution: [0. 0. 0. 0. 0.]
5x5 Relative residual: 1.0
5x5 Relative error vs NumPy: 1.0


In [ ]:
#1.2
# exact solution
x_star = np.array([0., 1.])

# loop over eps values
for k in range(1, 17):
    eps = 10.0**(-k)
    A = np.array([[1., eps],
                  [1., 1.]])
    b = np.array([1.+eps, 2.])

    # with pivoting
    x_piv, relres_piv = LU.LUsolve(A, b, pivoting=True)
    err_piv = npla.norm(x_piv - x_star)

    # without pivoting
    x_nopiv, relres_nopiv = LU.LUsolve(A, b, pivoting=False)
    err_nopiv = npla.norm(x_nopiv - x_star)

    print(f"eps = 1e-{k}")
    print(" with pivoting:   x =", x_piv, " error =", err_piv, " relres =", relres_piv)
    print(" no pivoting:     x =", x_nopiv, " error =", err_nopiv, " relres =", relres_nopiv)

    #our results from LU are approximations which

eps = 1e-1
 with pivoting:   x = [1. 1.]  error = 1.0  relres = 0.0
 no pivoting:     x = [1. 1.]  error = 1.0  relres = 0.0
eps = 1e-2
 with pivoting:   x = [1. 1.]  error = 1.0  relres = 0.0
 no pivoting:     x = [1. 1.]  error = 1.0  relres = 0.0
eps = 1e-3
 with pivoting:   x = [1. 1.]  error = 0.9999999999999999  relres = 0.0
 no pivoting:     x = [1. 1.]  error = 0.9999999999999999  relres = 0.0
eps = 1e-4
 with pivoting:   x = [1. 1.]  error = 1.0  relres = 0.0
 no pivoting:     x = [1. 1.]  error = 1.0  relres = 0.0
eps = 1e-5
 with pivoting:   x = [1. 1.]  error = 1.0  relres = 0.0
 no pivoting:     x = [1. 1.]  error = 1.0  relres = 0.0
eps = 1e-6
 with pivoting:   x = [1. 1.]  error = 0.9999999999999999  relres = 0.0
 no pivoting:     x = [1. 1.]  error = 0.9999999999999999  relres = 0.0
eps = 1e-7
 with pivoting:   x = [1. 1.]  error = 1.0  relres = 0.0
 no pivoting:     x = [1. 1.]  error = 1.0  relres = 0.0
eps = 1e-8
 with pivoting:   x = [1. 1.]  error = 0.9999999999999

In [ ]:
#3
import numpy as np
import numpy.linalg as npla
import scipy.sparse as sp

def GSsolve(A, b, tol=1e-8, max_iters=1000, callback=None):
    """
    Gauss–Seidel for Ax = b. Same behavior as Jsolve/CGsolve:
    - x starts at zeros
    - rel_res[0] = 1.0
    - stops when ||b - A x||/||b|| <= tol or hits max_iters
    """
    # make sure CSR
    A = sp.csr_matrix(A)
    n = A.shape[0]
    x = np.zeros(n, dtype=float)

    # precompute diagonal
    d = A.diagonal()
    if np.any(d == 0):
        raise ValueError("zero entry on diagonal; GS needs nonzero diag")

    # track relative residuals (like your solvers)
    rel_res = [1.0]
    bnorm = npla.norm(b)

    if callback is not None:
        callback(x=x, iteration=0, residual=1.0)

    # iterations
    for k in range(1, max_iters+1):
        # forward sweep (in-place updates)
        for i in range(n):
            start, end = A.indptr[i], A.indptr[i+1]
            row_idx = A.indices[start:end]
            row_val = A.data[start:end]

            # sum_{j != i} a_ij * x_j   (x_j uses newest values for j<i)
            s = 0.0
            for jj, aij in zip(row_idx, row_val):
                if jj != i:
                    s += aij * x[jj]
            x[i] = (b[i] - s) / d[i]

        # residual and stop
        this_rel = npla.norm(b - A @ x) / bnorm
        rel_res.append(this_rel)

        if callback is not None:
            callback(x=x, iteration=k, residual=this_rel)

        if this_rel <= tol:
            break

    return x, rel_res
